In [1]:
from zipfile import ZipFile
import requests
from io import BytesIO
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image


D:\Anaconda\envs\Test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\Test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\Test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\Test\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

In [2]:
dataset_link = 'https://github.com/andreianmatos/temporal_spaces_image_similarities/raw/main/people.zip'

response = requests.get(dataset_link)
zip_data = BytesIO(response.content)
with ZipFile(zip_data, 'r') as zip_ref:
    zip_ref.extractall()

In [8]:
image_link = 'https://github.com/modina-eu/temporal_spaces_image_similarities/raw/main/current_person.png'
response = requests.get(image_link)
query_image_data = BytesIO(response.content)
query_image = Image.open(query_image_data)

In [5]:
import requests
from io import BytesIO
import imageio

image_link = 'https://github.com/modina-eu/temporal_spaces_image_similarities/blob/main/current_person.png'

response = requests.get(image_link)

if response.status_code == 200:
    image_data = BytesIO(response.content)
    try:
        query_image = imageio.imread(image_data)
        print("Image loaded successfully.")
    except Exception as e:
        print("Error loading image:", e)
else:
    print("Failed to fetch image:", response.status_code)

ModuleNotFoundError: No module named 'imageio'

In [9]:
base_model = ResNet50(weights='imagenet', include_top=False)


D:\Anaconda\envs\Test\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 15s 0us/step


In [10]:
def get_image_embedding(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    embeddings = base_model.predict(img_array)
    return embeddings.flatten()

In [11]:
def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    similarity = dot_product / (norm1 * norm2)
    return similarity

In [12]:
query_embedding = get_image_embedding(query_image)

most_similar_image_path = None
max_similarity = -1

for root, dirs, files in os.walk('people/'):
    for file in files:
        if file.lower().endswith(('.png')):
            img_path = os.path.join(root, file)
            img_embedding = get_image_embedding(img_path)
            similarity_score = cosine_similarity(query_embedding, img_embedding)

            if similarity_score > max_similarity:
                max_similarity = similarity_score
                most_similar_image_path = img_path

TypeError: expected str, bytes or os.PathLike object, not PngImageFile